This is the Jupyter Notebook that contains analyzing work for my League of Legends prediction bracket. 

In [112]:
# Import statements
import pandas as pd # database loading and manipulation library
import numpy as np # numeric python lybrary
import matplotlib.pyplot as plt # plotting and graphing library
import operator
%matplotlib notebook 

In [4]:
# Importing the csv file from Oracle.
summer = pd.read_csv('2019SummerMatchDataOraclesElixir.csv')
spring = pd.read_csv('2019SpringMatchDataOraclesElixir.csv')
worlds = pd.read_csv('2019WorldsMatchDataOraclesElixir.csv')

/Users/main/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/main/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#summer.head()
#spring.head()
#worlds.head()

In [6]:
summer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12612 entries, 0 to 12611
Data columns (total 98 columns):
gameid                     12612 non-null object
url                        8676 non-null object
league                     12612 non-null object
split                      12612 non-null object
date                       12612 non-null object
week                       12612 non-null object
game                       12612 non-null object
patchno                    12612 non-null float64
playerid                   12612 non-null int64
side                       12612 non-null object
position                   12612 non-null object
player                     12612 non-null object
team                       12612 non-null object
champion                   12612 non-null object
ban1                       12612 non-null object
ban2                       12612 non-null object
ban3                       12588 non-null object
ban4                       12582 non-null object
ban5      

In [ ]:
spring.info()

In [1]:
worlds.info()

NameError: name 'worlds' is not defined

In [10]:
# Rows to remove
cleanSummer = summer.drop(['url','date','week','game','patchno','gameid','split'],axis = 1)
cleanSpring = spring.drop(['url','date','week','game','patchno','gameid','split'],axis = 1)

Data from World's is irrelevant since that is the actual event. It'll be relevant for quarterfinals and up, but not groups.

In [19]:
cleanSummer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12612 entries, 0 to 12611
Data columns (total 91 columns):
league                     12612 non-null object
playerid                   12612 non-null int64
side                       12612 non-null object
position                   12612 non-null object
player                     12612 non-null object
team                       12612 non-null object
champion                   12612 non-null object
ban1                       12612 non-null object
ban2                       12612 non-null object
ban3                       12588 non-null object
ban4                       12582 non-null object
ban5                       12582 non-null object
gamelength                 12612 non-null float64
result                     12612 non-null int64
k                          12612 non-null int64
d                          12612 non-null int64
a                          12612 non-null int64
teamkills                  12612 non-null int64
teamdeaths    

In [171]:
def CreepRatingScore(data):
  data['CSRate']=data['minionkills']+data['monsterkills']
  
def CSvsOpp(data):
  data['CSvOPP']=data['gdat10']+data['gdat15']

def GoldvOpp(data):
  data['GoldvOPP']=data['csdat10']+data['csdat15']

def GoldScore(data):
  data['GoldScore']=data['totalgold']+data['earnedgpm']+data['goldspent']+data['gspd']
  
def WardScore(data):
  data['WardScore']=data['wards']+data['wpm']+data['wardshare']+data['wardkills']+data['visionwards']+data['visionwardbuys']

def KDAScore(data):
  data['KDA']=data['k']+data['d']+data['a']
  
def JungleRating(data):
  data['JungleRate']=data['monsterkillsownjungle']+data['monsterkillsenemyjungle']
  
def XPDiff(data):
  data['XPDiff']=data['xpdat10']
  
def TimeScores(data):
  data['tScores'] = data['csdat10']+data['csdat15']+data['gdat10']+data['gdat15']+data['xpdat10']
  
data['earnedgpm']+data['cspm']+data['wpm']+data['dmgtochampsperminute']+

In [131]:
# Essentially we created an overloaded number here, so we'll divide by the amount of games each team played later
def PlayerScore(data):
  data['PlayerScore']=data['KDA']+data['JungleRate']+data['WardScore']+data['GoldvOPP']+data['CSvOPP']
  +data['CSRate']+data['XPDiff']
  +(data['GoldScore']/100)+data['teamdragkills']
  +data['fd']+data['fbaron']
  +data['result']*1000+data['dmgtochamps']

In [132]:
data2=cleanSummer
data2.fillna(0)

,league,playerid,side,position,player,team,champion,ban1,ban2,ban3,...,csdat15,CSRate,CSvOPP,GoldvOPP,GoldScore,WardScore,KDA,JungleRate,XPDiff,PlayerScore
0,LCS,1,Blue,Top,Licorice,Cloud9,Aatrox,Draven,Rakan,Rek'Sai,...,-25.0,292.000000,-1205.0,-42.0,30747.951692,47.821553,21,17.0,-805.0,-1161.178447
1,LCS,2,Blue,Jungle,Svenskeren,Cloud9,Hecarim,Draven,Rakan,Rek'Sai,...,17.0,205.633803,807.0,26.0,25328.009928,42.410777,16,110.0,621.0,1001.410777
2,LCS,3,Blue,Middle,Nisqy,Cloud9,Sylas,Draven,Rakan,Rek'Sai,...,-6.0,292.000000,928.0,-9.0,31521.581474,24.373433,18,28.0,424.0,989.373433
3,LCS,4,Blue,ADC,Sneaky,Cloud9,Varus,Draven,Rakan,Rek'Sai,...,26.0,299.197183,882.0,54.0,29452.611699,35.448120,16,20.0,273.0,1007.448120
4,LCS,5,Blue,Support,Zeyzal,Cloud9,Galio,Draven,Rakan,Rek'Sai,...,1.0,71.971831,-59.0,2.0,18373.889654,106.016540,16,0.0,126.0,65.016540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12607,LEC,8,Red,Middle,Humanoid,Splyce,Yasuo,Yuumi,Akali,Aatrox,...,2.0,268.396239,-1194.0,9.0,20201.523940,15.503285,9,6.0,82.0,-1154.496715
12608,LEC,9,Red,ADC,Kobbe,Splyce,Kai'Sa,Yuumi,Akali,Aatrox,...,-27.0,246.550034,-1584.0,-37.0,21199.079368,10.335523,11,31.0,-1027.0,-1568.664477
12609,LEC,10,Red,Support,Norskeren,Splyce,Shen,Yuumi,Akali,Aatrox,...,1.0,49.934184,-850.0,1.0,11444.907662,56.677615,10,0.0,216.0,-782.322385
12610,LEC,100,Blue,Team,Team,Fnatic,,Qiyana,Gragas,Rakan,...,14.0,848.881128,2803.0,24.0,100762.140985,164.022163,72,115.0,1147.0,3178.022163


In [133]:
CreepRatingScore(data2)
CSvsOpp(data2)
GoldvOpp(data2)
GoldScore(data2)
WardScore(data2)
KDAScore(data2)
JungleRating(data2)
XPDiff(data2)

# Reports an error in the calculation, but still computes the formula correctly. Unsure of why that is.
PlayerScore(data2)

In [134]:
# Reports an error in the calculation, but still computes the formula correctly. Unsure of why that is.
PlayerScore(data2)

TypeError: can only concatenate str (not "float") to str

In [135]:
Scores = {}

Cloud9TeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Cloud9')]
Cloud9Team = Cloud9TeamScores.sum()
C9Score = Cloud9Team['PlayerScore']
Scores['C9'] = C9Score / len(Cloud9TeamScores)

TLTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Team Liquid')]
TLTeam = TLTeamScores.sum()
TLScore = TLTeam['PlayerScore']
Scores['TL'] = TLScore / len(TLTeamScores)

CGTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Clutch Gaming')]
CGTeam = CGTeamScores.sum()
CGScore = CGTeam['PlayerScore']
Scores['CG'] = CGScore / len(CGTeamScores)

G2TeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'G2 Esports')]
G2Team = G2TeamScores.sum()
G2Score = G2Team['PlayerScore']
Scores['G2'] = G2Score / len(G2TeamScores)

FNCTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Fnatic')]
FNCTeam = FNCTeamScores.sum()
FNCScore = FNCTeam['PlayerScore']
Scores['FNC'] = FNCScore / len(FNCTeamScores)

SpyTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Splyce')]
SpyTeam = SpyTeamScores.sum()
SpyScore = SpyTeam['PlayerScore']
Scores['Spy'] = SpyScore / len(SpyTeamScores)

FPXTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Funplus Phoenix')]
FPXTeam = FPXTeamScores.sum()
FPXScore = FPXTeam['PlayerScore']
Scores['FPX'] = FPXScore / len(FPXTeamScores)

RNGTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Royal Never Give Up')]
RNGTeam = RNGTeamScores.sum()
RNGScore = RNGTeam['PlayerScore']
Scores['RNG'] = RNGScore / len(RNGTeamScores)

IGTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Invictus Gaming')]
IGTeam = IGTeamScores.sum()
IGScore = IGTeam['PlayerScore']
Scores['IG'] = IGScore / len(IGTeamScores)

SKTTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'SK Telecom T1')]
SKTTeam = SKTTeamScores.sum()
SKTScore = SKTTeam['PlayerScore']
Scores['SKT'] = SKTScore / len(SKTTeamScores)

GRFTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Griffin')]
GRFTeam = GRFTeamScores.sum()
GRFScore = GRFTeam['PlayerScore']
Scores['GRF'] = GRFScore / len(GRFTeamScores)

DWGTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Damwon Gaming')]
DWGTeam = DWGTeamScores.sum()
DWGScore = DWGTeam['PlayerScore']
Scores['DWG'] = DWGScore / len(DWGTeamScores)

AHQTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'AHQ e-Sports Club')]
AHQTeam = AHQTeamScores.sum()
AHQScore = AHQTeam['PlayerScore']
Scores['AHQ'] = AHQScore / len(AHQTeamScores)

JTeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'J Team')]
JTeam = JTeamScores.sum()
JScore = JTeam['PlayerScore']
Scores['JTeam'] = JScore / len(JTeamScores)

HKATeamScores = data2.loc[(data2['player'] == 'Team') & (data2['team'] == 'Hong Kong Attitude')]
HKATeam = HKATeamScores.sum()
HKAScore = HKATeam['PlayerScore']
Scores['HKA'] = HKAScore / len(HKATeamScores)

# Missing GAM

sorted_Scores = sorted(Scores.items(), key=operator.itemgetter(1), reverse = True)
sorted_Scores

[('JTeam', 3001.407218963657),
 ('G2', 2871.17031023175),
 ('GRF', 2427.517261463195),
 ('TL', 1932.7435486360005),
 ('C9', 1596.3031551292145),
 ('FNC', 1335.1759740906764),
 ('CG', 1314.3508213260445),
 ('AHQ', 1222.3240909242309),
 ('Spy', 1008.3077848510625),
 ('SKT', 977.5392282498572),
 ('DWG', 821.161870378577),
 ('HKA', 781.2986142861591),
 ('FPX', 0.0),
 ('RNG', 0.0),
 ('IG', 0.0)]

* LMS 1
* LEC 1
* LCK 2
* LCS 1
* LCS 2
* LEC 2
* LCS 3
* LMS 2
* LEC 3
* LCK 1
* LCK 3
* LMS 3
* LPL 1
* LPL 2
* LPL 3
* VCS 1



* Griffin
* Funplus Phoenix
* SKT Telecom T1
* Royal Never Give Up
* Damwon Gaming
* Invictus Gaming
* G2 Esports
* Team Liquid
* Fnatic
* Cloud 9
* Splyce
* Clutch Gaming
* J Team
* AHQ e-Sports Club
* Hong Kong Attitude
* GAM Esports

In [143]:
Scores = {}

Cloud9TeamScores = data2.loc[(data2['team'] == 'Cloud9')]
Cloud9Team = Cloud9TeamScores.sum()
C9Score = Cloud9Team['PlayerScore']
Scores['C9'] = C9Score / len(Cloud9TeamScores)

TLTeamScores = data2.loc[(data2['team'] == 'Team Liquid')]
TLTeam = TLTeamScores.sum()
TLScore = TLTeam['PlayerScore']
Scores['TL'] = TLScore / len(TLTeamScores)

CGTeamScores = data2.loc[(data2['team'] == 'Clutch Gaming')]
CGTeam = CGTeamScores.sum()
CGScore = CGTeam['PlayerScore']
Scores['CG'] = CGScore / len(CGTeamScores)

G2TeamScores = data2.loc[(data2['team'] == 'G2 Esports')]
G2Team = G2TeamScores.sum()
G2Score = G2Team['PlayerScore']
Scores['G2'] = G2Score / len(G2TeamScores)

FNCTeamScores = data2.loc[(data2['team'] == 'Fnatic')]
FNCTeam = FNCTeamScores.sum()
FNCScore = FNCTeam['PlayerScore']
Scores['FNC'] = FNCScore / len(FNCTeamScores)

SpyTeamScores = data2.loc[(data2['team'] == 'Splyce')]
SpyTeam = SpyTeamScores.sum()
SpyScore = SpyTeam['PlayerScore']
Scores['Spy'] = SpyScore / len(SpyTeamScores)

FPXTeamScores = data2.loc[(data2['team'] == 'Funplus Phoenix')]
FPXTeam = FPXTeamScores.sum()
FPXScore = FPXTeam['PlayerScore']
Scores['FPX'] = FPXScore / len(FPXTeamScores)

RNGTeamScores = data2.loc[(data2['team'] == 'Royal Never Give Up')]
RNGTeam = RNGTeamScores.sum()
RNGScore = RNGTeam['PlayerScore']
Scores['RNG'] = RNGScore / len(RNGTeamScores)

IGTeamScores = data2.loc[(data2['team'] == 'Invictus Gaming')]
IGTeam = IGTeamScores.sum()
IGScore = IGTeam['PlayerScore']
Scores['IG'] = IGScore / len(IGTeamScores)

SKTTeamScores = data2.loc[(data2['team'] == 'SK Telecom T1')]
SKTTeam = SKTTeamScores.sum()
SKTScore = SKTTeam['PlayerScore']
Scores['SKT'] = SKTScore / len(SKTTeamScores)

GRFTeamScores = data2.loc[(data2['team'] == 'Griffin')]
GRFTeam = GRFTeamScores.sum()
GRFScore = GRFTeam['PlayerScore']
Scores['GRF'] = GRFScore / len(GRFTeamScores)

DWGTeamScores = data2.loc[(data2['team'] == 'Damwon Gaming')]
DWGTeam = DWGTeamScores.sum()
DWGScore = DWGTeam['PlayerScore']
Scores['DWG'] = DWGScore / len(DWGTeamScores)

AHQTeamScores = data2.loc[(data2['team'] == 'AHQ e-Sports Club')]
AHQTeam = AHQTeamScores.sum()
AHQScore = AHQTeam['PlayerScore']
Scores['AHQ'] = AHQScore / len(AHQTeamScores)

JTeamScores = data2.loc[(data2['team'] == 'J Team')]
JTeam = JTeamScores.sum()
JScore = JTeam['PlayerScore']
Scores['JTeam'] = JScore / len(JTeamScores)

HKATeamScores = data2.loc[(data2['team'] == 'Hong Kong Attitude')]
HKATeam = HKATeamScores.sum()
HKAScore = HKATeam['PlayerScore']
Scores['HKA'] = HKAScore / len(HKATeamScores)

# Missing GAM

sorted_Scores = sorted(Scores.items(), key=operator.itemgetter(1), reverse = True)
sorted_Scores

[('JTeam', 1000.4690729879239),
 ('G2', 957.0567700772441),
 ('GRF', 809.1724204877319),
 ('TL', 644.2478495453215),
 ('C9', 532.1010517097142),
 ('FNC', 445.05865803019594),
 ('CG', 438.1169404420481),
 ('AHQ', 407.4413636414017),
 ('Spy', 336.10259495034376),
 ('SKT', 325.846409416625),
 ('DWG', 273.72062345954487),
 ('HKA', 260.4328714286742),
 ('FPX', 0.0),
 ('RNG', 0.0),
 ('IG', 0.0)]

In [149]:
cleanWorlds = worlds.drop(['url','date','game','patchno','gameid','split'],axis = 1)
data3=cleanWorlds
data3.fillna(0)
data4 = data3.loc[(data3['week'] == 'G')]
data4

,league,week,playerid,side,position,player,team,champion,ban1,ban2,...,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
504,WC,G,1,Blue,Top,Bwipo,Fnatic,Gangplank,Elise,Qiyana,...,231,4534,4728,-194,79,82,-3,126,140,-14
505,WC,G,2,Blue,Jungle,Broxah,Fnatic,Rek'Sai,Elise,Qiyana,...,-935,3113,3036,77,57,54,3,84,97,-13
506,WC,G,3,Blue,Middle,Nemesis,Fnatic,Ryze,Elise,Qiyana,...,-562,4682,4686,-4,93,86,7,152,138,14
507,WC,G,4,Blue,ADC,Rekkles,Fnatic,Garen,Elise,Qiyana,...,-388,2897,3271,-374,82,74,8,139,135,4
508,WC,G,5,Blue,Support,Hylissang,Fnatic,Yuumi,Elise,Qiyana,...,976,2543,2329,214,16,16,0,34,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,WC,G,8,Red,Middle,Jensen,Team Liquid,LeBlanc,Pantheon,Jayce,...,-615,4203,4571,-368,78,71,7,128,106,22
1100,WC,G,9,Red,ADC,Doublelift,Team Liquid,Xayah,Pantheon,Jayce,...,-127,3383,3607,-224,93,97,-4,148,145,3
1101,WC,G,10,Red,Support,CoreJJ,Team Liquid,Nautilus,Pantheon,Jayce,...,188,2853,2268,585,15,16,-1,23,24,-1
1102,WC,G,100,Blue,Team,Team,Invictus Gaming,,Jarvan IV,Rakan,...,2841,18918,19064,-146,335,321,14,507,490,17


In [292]:
CreepRatingScore(data4)
CSvsOpp(data4)
GoldvOpp(data4)
GoldScore(data4)
WardScore(data4)
KDAScore(data4)
JungleRating(data4)
XPDiff(data4)

#def PlayerScore(data):
 # data['PlayerScore']=data['KDA']+data['JungleRate']*100+data['WardScore']+data['GoldvOPP']+data['CSvOPP']+data['CSRate']+data['XPDiff']+(data['GoldScore']/1000)+data['teamdragkills']+data['fd']+data['fbaron']+data['dmgtochamps']+data['result']*100
def TimeScores(data):
  data['tScores'] = (data['csdat10']+data['csdat15'])*50+data['gdat10']+data['gdat15']+data['xpdat10']*.1
  
def PlayerScoreTime(data):
  data['PlayerScore']=data['KDA']+(data['fb']+data['fbassist'])*500+data['earnedgpm']*2+data['cspm']*3+data['wpm']+data['wcpm']*100+data['dmgtochampsperminute']*3.4+data['teamdragkills']*20+data['fd']*100+data['fbaron']*20+(data['tScores'])
# Reports an error in the calculation, but still computes the formula correctly. Unsure of why that is.
TimeScores(data4)
PlayerScoreTime(data4)

/Users/main/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/main/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/main/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [293]:
GScores = {}

G2TeamScores = data4.loc[(data4['team'] == 'G2 Esports')]
G2Team = G2TeamScores.sum()
G2Score = G2Team['PlayerScore']
GScores['G2'] = G2Score / len(G2TeamScores)

FNCTeamScores = data4.loc[(data4['team'] == 'Fnatic')]
FNCTeam = FNCTeamScores.sum()
FNCScore = FNCTeam['PlayerScore']
GScores['FNC'] = FNCScore / len(FNCTeamScores)

SpyTeamScores = data4.loc[(data4['team'] == 'Splyce')]
SpyTeam = SpyTeamScores.sum()
SpyScore = SpyTeam['PlayerScore']
GScores['Spy'] = SpyScore / len(SpyTeamScores)

FPXTeamScores = data4.loc[(data4['team'] == 'Funplus Phoenix')]
FPXTeam = FPXTeamScores.sum()
FPXScore = FPXTeam['PlayerScore']
GScores['FPX'] = FPXScore / len(FPXTeamScores)

IGTeamScores = data4.loc[(data4['team'] == 'Invictus Gaming')]
IGTeam = IGTeamScores.sum()
IGScore = IGTeam['PlayerScore']
GScores['IG'] = IGScore / len(IGTeamScores)

SKTTeamScores = data4.loc[(data4['team'] == 'SK Telecom T1')]
SKTTeam = SKTTeamScores.sum()
SKTScore = SKTTeam['PlayerScore']
GScores['SKT'] = SKTScore / len(SKTTeamScores)

GRFTeamScores = data4.loc[(data4['team'] == 'Griffin')]
GRFTeam = GRFTeamScores.sum()
GRFScore = GRFTeam['PlayerScore']
GScores['GRF'] = GRFScore / len(GRFTeamScores)

DWGTeamScores = data4.loc[(data4['team'] == 'Damwon Gaming')]
DWGTeam = DWGTeamScores.sum()
DWGScore = DWGTeam['PlayerScore']
GScores['DWG'] = DWGScore / len(DWGTeamScores)

sorted_GScores = sorted(GScores.items(), key=operator.itemgetter(1), reverse = True)
sorted_GScores

[('FPX', 5076.418541646548),
 ('GRF', 4578.536910618024),
 ('G2', 4470.039316664047),
 ('DWG', 4373.696768487),
 ('SKT', 4357.350642633167),
 ('IG', 3316.710902169333),
 ('Spy', 3108.4106676135716),
 ('FNC', 2590.061620456944)]

Rerun Summer Ratings to see if we get a difference

In [297]:
data5=cleanSummer
data5.fillna(0)

TimeScores(data5)
PlayerScoreTime(data5)

TypeError: can only concatenate str (not "float") to str

In [298]:
Scores = {}

Cloud9TeamScores = data5.loc[(data5['team'] == 'Cloud9')]
Cloud9Team = Cloud9TeamScores.sum()
C9Score = Cloud9Team['PlayerScore']
Scores['C9'] = C9Score / len(Cloud9TeamScores)

TLTeamScores = data5.loc[(data5['team'] == 'Team Liquid')]
TLTeam = TLTeamScores.sum()
TLScore = TLTeam['PlayerScore']
Scores['TL'] = TLScore / len(TLTeamScores)

CGTeamScores = data5.loc[(data5['team'] == 'Clutch Gaming')]
CGTeam = CGTeamScores.sum()
CGScore = CGTeam['PlayerScore']
Scores['CG'] = CGScore / len(CGTeamScores)

G2TeamScores = data5.loc[(data5['team'] == 'G2 Esports')]
G2Team = G2TeamScores.sum()
G2Score = G2Team['PlayerScore']
Scores['G2'] = G2Score / len(G2TeamScores)

FNCTeamScores = data5.loc[(data5['team'] == 'Fnatic')]
FNCTeam = FNCTeamScores.sum()
FNCScore = FNCTeam['PlayerScore']
Scores['FNC'] = FNCScore / len(FNCTeamScores)

SpyTeamScores = data5.loc[(data5['team'] == 'Splyce')]
SpyTeam = SpyTeamScores.sum()
SpyScore = SpyTeam['PlayerScore']
Scores['Spy'] = SpyScore / len(SpyTeamScores)

SKTTeamScores = data5.loc[(data5['team'] == 'SK Telecom T1')]
SKTTeam = SKTTeamScores.sum()
SKTScore = SKTTeam['PlayerScore']
Scores['SKT'] = SKTScore / len(SKTTeamScores)

GRFTeamScores = data5.loc[(data5['team'] == 'Griffin')]
GRFTeam = GRFTeamScores.sum()
GRFScore = GRFTeam['PlayerScore']
Scores['GRF'] = GRFScore / len(GRFTeamScores)

DWGTeamScores = data5.loc[(data5['team'] == 'Damwon Gaming')]
DWGTeam = DWGTeamScores.sum()
DWGScore = DWGTeam['PlayerScore']
Scores['DWG'] = DWGScore / len(DWGTeamScores)

AHQTeamScores = data5.loc[(data5['team'] == 'AHQ e-Sports Club')]
AHQTeam = AHQTeamScores.sum()
AHQScore = AHQTeam['PlayerScore']
Scores['AHQ'] = AHQScore / len(AHQTeamScores)

JTeamScores = data5.loc[(data5['team'] == 'J Team')]
JTeam = JTeamScores.sum()
JScore = JTeam['PlayerScore']
Scores['JTeam'] = JScore / len(JTeamScores)

HKATeamScores = data5.loc[(data5['team'] == 'Hong Kong Attitude')]
HKATeam = HKATeamScores.sum()
HKAScore = HKATeam['PlayerScore']
Scores['HKA'] = HKAScore / len(HKATeamScores)

# Missing GAM

sorted_Scores = sorted(Scores.items(), key=operator.itemgetter(1), reverse = True)
sorted_Scores

[('JTeam', 1000.4690729879239),
 ('G2', 957.0567700772441),
 ('GRF', 809.1724204877319),
 ('TL', 644.2478495453215),
 ('C9', 532.1010517097142),
 ('FNC', 445.05865803019594),
 ('CG', 438.1169404420481),
 ('AHQ', 407.4413636414017),
 ('Spy', 336.10259495034376),
 ('SKT', 325.846409416625),
 ('DWG', 273.72062345954487),
 ('HKA', 260.4328714286742)]

In [312]:
worlds2018 = pd.read_csv('2018Worlds.csv')
cleanWorlds2018 = worlds2018.drop(['url','date','game','patchno','gameid','split'],axis = 1)
data6=cleanWorlds2018
data6.fillna(0)
data7 = data6.loc[(data3['week'] == 'G')]
data7

,league,week,playerid,side,position,player,team,champion,ban1,ban2,...,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
504,WC,G,1,Blue,Top,Smeb,KT Rolster,Aatrox,Sion,Tahm Kench,...,430,4535,4769,-234,87,84,3,150,135,15
505,WC,G,2,Blue,Jungle,Score,KT Rolster,Kindred,Sion,Tahm Kench,...,290,3675,2929,746,64,44,20,95,81,14
506,WC,G,3,Blue,Middle,Ucal,KT Rolster,Syndra,Sion,Tahm Kench,...,1051,5149,4146,1003,94,72,22,156,132,24
507,WC,G,4,Blue,ADC,Deft,KT Rolster,Xayah,Sion,Tahm Kench,...,-279,3188,3291,-103,91,78,13,137,135,2
508,WC,G,5,Blue,Support,Mata,KT Rolster,Thresh,Sion,Tahm Kench,...,-899,2509,3108,-599,13,3,10,19,7,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,WC,G,8,Red,Middle,Caps,Fnatic,Aatrox,Akali,Irelia,...,43,4864,4619,245,66,89,-23,104,142,-38
1100,WC,G,9,Red,ADC,Rekkles,Fnatic,Tristana,Akali,Irelia,...,47,3417,3337,80,88,96,-8,147,148,-1
1101,WC,G,10,Red,Support,Hylissang,Fnatic,Braum,Akali,Irelia,...,41,2776,2778,-2,16,14,2,29,25,4
1102,WC,G,100,Blue,Team,Team,Invictus Gaming,,Swain,Tahm Kench,...,-632,19457,19508,-51,346,309,37,542,516,26


In [313]:
data7['team'].unique()

array(['KT Rolster', 'Team Liquid', 'Edward Gaming', 'MAD Team',
       'Phong Vu Buffalo', 'Flash Wolves', 'Afreeca Freecs', 'G2 Esports',
       'Royal Never Give Up', 'Cloud9', 'Gen.G', 'Vitality',
       '100 Thieves', 'Fnatic', 'Invictus Gaming', 'G-Rex'], dtype=object)

In [314]:
KDAScore(data7)
TimeScores(data7)
PlayerScoreTime(data7)

/Users/main/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/main/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/main/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [318]:
G2018Scores = {}

G2TeamScores = data7.loc[(data7['team'] == 'G2 Esports')]
G2Team = G2TeamScores.sum()
G2Score = G2Team['PlayerScore']
G2018Scores['G2'] = G2Score / len(G2TeamScores)

FNCTeamScores = data7.loc[(data7['team'] == 'Fnatic')]
FNCTeam = FNCTeamScores.sum()
FNCScore = FNCTeam['PlayerScore']
G2018Scores['FNC'] = FNCScore / len(FNCTeamScores)

Cloud9Scores = data7.loc[(data7['team'] == 'Cloud9')]
C9Team = Cloud9Scores.sum()
C9Score = C9Team['PlayerScore']
G2018Scores['C9'] = C9Score / len(Cloud9Scores)

AFTeamScores = data7.loc[(data7['team'] == 'Afreeca Freecs')]
AFTeam = AFTeamScores.sum()
AFScore = AFTeam['PlayerScore']
G2018Scores['AF'] = AFScore / len(AFTeamScores)

IGTeamScores = data7.loc[(data7['team'] == 'Invictus Gaming')]
IGTeam = IGTeamScores.sum()
IGScore = IGTeam['PlayerScore']
G2018Scores['IG'] = IGScore / len(IGTeamScores)

KTTeamScores = data7.loc[(data7['team'] == 'KT Rolster')]
KTTeam = KTTeamScores.sum()
KTScore = KTTeam['PlayerScore']
G2018Scores['KT'] = KTScore / len(KTTeamScores)

RNGTeamScores = data7.loc[(data7['team'] == 'Royal Never Give Up')]
RNGTeam = GRFTeamScores.sum()
RNGScore = GRFTeam['PlayerScore']
G2018Scores['RNG'] = GRFScore / len(GRFTeamScores)

EDGTeamScores = data7.loc[(data7['team'] == 'Edward Gaming')]
EDGTeam = EDGTeamScores.sum()
EDGScore = EDGTeam['PlayerScore']
G2018Scores['EDG'] = EDGScore / len(EDGTeamScores)

sorted_G2018Scores = sorted(G2018Scores.items(), key=operator.itemgetter(1), reverse = True)
sorted_G2018Scores

[('IG', 4918.989508895888),
 ('FNC', 4770.383770918388),
 ('KT', 4351.8161867973595),
 ('G2', 3460.6896921934767),
 ('C9', 2957.315279118572),
 ('EDG', 2714.1016108385547),
 ('AF', 2342.283303995499),
 ('RNG', 809.1724204877319)]